In [2]:
%load_ext memory_profiler

In [1]:
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
%%memit
cust_week = pd.read_csv("./cust_week.csv")

peak memory: 1344.94 MiB, increment: 1241.75 MiB


In [4]:
# Num of active weeks
cust_week.groupby("customer_id").size().describe()

count    1.362281e+06
mean     5.937411e+00
std      7.660700e+00
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      8.000000e+00
max      1.040000e+02
dtype: float64

In [5]:
toy = cust_week.query("customer_id=='ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7'")

In [6]:
toy

,customer_id,week,0
8088383,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,38,[671695001]
8088384,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,46,"[562245015, 562245018, 562245004]"
8088385,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,50,[657924001]
8088386,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,62,"[625939035, 624486001, 702404001, 624486003]"
8088387,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,69,"[740652003, 700851002, 676979007, 507909001, 7..."
8088388,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,82,"[703160006, 685811001, 624486028, 624486051, 6..."
8088389,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,85,"[759871004, 759871002, 759871001]"
8088390,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,92,"[809025002, 660308001, 681373016, 674606037, 7..."
8088391,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,99,"[706016025, 706016028, 702208002, 706016015, 7..."
8088392,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,106,"[818754003, 624486010, 699623004, 759871013]"


In [11]:
def get_seq(df):
    import pandas as pd
    wk_seq = df.apply(lambda t: [str(t["week"])]*len(eval(t["0"])), axis=1).sum()
    item_seq = df["0"].apply(lambda s: [str(i) for i in eval(s)]).sum()
    
    return pd.DataFrame({"wk": [wk_seq], "item": [item_seq]})

In [8]:
def get_wk_seq(df):
    wk_seq = df.apply(lambda t: [t["week"]]*len(eval(t["0"])), axis=1).sum()
    
    return wk_seq
def get_item_seq(df):
    item_seq = df["0"].apply(eval).sum()
    return item_seq

In [9]:
%%time
_=cust_week.iloc[:100000, :].groupby("customer_id").apply(get_seq,);

Wall time: 13.9 s


In [16]:
%%time
_=cust_week.iloc[:100000, :].groupby("customer_id").parallel_apply(get_seq);

Wall time: 6.15 s


In [23]:
%%time
cust_week_seq = cust_week.groupby("customer_id").parallel_apply(get_seq);

Wall time: 10min 53s


In [24]:
cust_week_seq.head()

,,wk,item
customer_id,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0,"[51, 51, 51, 69, 72, 72, 81, 81, 89, 90, 90, 9...","[625548001, 176209023, 627759010, 697138006, 5..."
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0,"[37, 37, 37, 38, 38, 38, 38, 38, 53, 66, 66, 6...","[583558001, 639677008, 640244003, 521269001, 6..."
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0,"[37, 37, 38, 60, 109, 109, 109, 117, 117, 119,...","[663713001, 541518023, 663713001, 578020002, 7..."
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0,"[74, 74]","[742079001, 732413001]"
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,0,"[40, 40, 61, 61, 91, 92, 126, 126, 126, 136, 1...","[634249005, 677049001, 698286003, 707704003, 3..."


In [25]:
cust_week_seq.to_hdf("./cust_week_seq.h5", key="df")

C:\Users\Francis\Python\python39\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['wk', 'item'], dtype='object')]

  pytables.to_hdf(


In [44]:
import torchtext

In [78]:
V = torchtext.vocab.build_vocab_from_iterator(
    cust_week_seq["item"][:10000].apply(lambda l: [str(i) for i in l]),
        min_freq=5, specials=["OOV"])

In [79]:
V.set_default_index(V.get_stoi()["OOV"])

In [83]:
%timeit V.lookup_indices([str(i) for i in cust_week_seq["item"][1]])

14.5 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
